In [ ]:
import numpy as np
import json
import matplotlib.pyplot as plt
from cellpack_analysis.utilities.PILR_tools import (
    average_over_dimension,
)

Read in individual PILRs

In [ ]:
base_path = "/allen/aics/animated-cell/Saurabh/cellpack-analysis/results/sample_8d/"
file_name = "individual_PILR.json"

In [ ]:
with open(base_path + file_name, "r") as f:
    individual_PILR_dict = json.load(f)

In [ ]:
individual_PILR_dict_bak = individual_PILR_dict.copy()

In [ ]:
individual_PILR_dict = individual_PILR_dict_bak.copy()

In [ ]:
average_over_phi = True

In [ ]:
for key, value in individual_PILR_dict.items():
    value = np.array(value)
    if average_over_phi:
        avg_value = np.zeros((value.shape[0], value.shape[1], 64))
        for ind in range(value.shape[0]):
            avg_value[ind] = average_over_dimension(value[ind])
        value = avg_value
    
    new_values = np.zeros((value.shape[0], value.shape[1] // 2, value.shape[2]))
    for ind in range(value.shape[0]):
        new_values[ind] = value[ind][(len(value[ind]) // 2 + 1) :]
        new_values[ind] = new_values[ind] / np.max(new_values[ind])
    individual_PILR_dict[key] = new_values.reshape(new_values.shape[0], -1)
    print(key, individual_PILR_dict[key].shape)

In [ ]:
combined_PILR = np.concatenate(list(individual_PILR_dict.values()), axis=0)
combined_PILR.shape

## Plot pacmap/pca

In [ ]:
from cellpack_analysis.utilities.PILR_tools import get_embeddings

In [ ]:
all_channel_list = list(individual_PILR_dict.keys())

In [ ]:
all_channel_list

In [ ]:
metric = "pca"
# channels_for_embedding = [ch for ch in all_channel_list if ch != "SLC25A17"]
channels_for_embedding = ["SLC25A17"]

In [ ]:
embedding_dict = get_embeddings(
    individual_PILR_dict,
    metric=metric,
    channels_for_embedding=channels_for_embedding,
    n_components_pca=8,
)

In [ ]:
for ch, ch_dict in embedding_dict.items():
    print(ch, ch_dict["embedding"])

### Plot colorized pacmap/pca

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(5, 5), dpi=300)
prev_ind = 0
for col_ind, (ch, ch_dict) in enumerate(embedding_dict.items()):
    embedding = ch_dict["embedding"]
    values = ch_dict["values"]
    ax.scatter(
        values[:, 0],
        values[:, 1],
        s=20,
        label=ch,
        alpha=0.5 if embedding == "fit" else 0.7,
        marker="o" if embedding == "fit" else "x",
        color=f"C{col_ind}",
        zorder=1 if embedding == "fit" else 2,
        edgecolors="none" if embedding == "fit" else None,
    )

ax.legend(bbox_to_anchor=(1.05, 0.5), loc="center left", borderaxespad=0.0)
ax.set_xlabel(f"{metric} 1")
ax.set_ylabel(f"{metric} 2")
plt.show()

## Plot phi averaged PILRs

In [ ]:
from cellpack_analysis.utilities.plotting_tools import save_PILR_image

In [ ]:
base_path = "/allen/aics/animated-cell/Saurabh/cellpack_analysis/results/sample_8d/"
file_name = "avg_PILR.json"

In [ ]:
with open(base_path + file_name, "r") as f:
    avg_PILR_dict = json.load(f)

In [ ]:
avg_PILR_dict_bak = avg_PILR_dict.copy()

In [ ]:
for ch, value in avg_PILR_dict.items():
    value = np.array(value)
    avg_PILR_dict[ch] = average_over_phi(value)

In [ ]:
result_folder = "../../results/sample_8d"

In [ ]:
for ch, value in avg_PILR_dict.items():
    save_PILR_image(value, ch, save_dir=result_folder, suffix="_avg_phi", aspect=1)

## 3D PILR analysis